In [1]:
import keyring
import pandas as pd
import datetime
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymongo import MongoClient

In [2]:
# Get dates for query
today = datetime.datetime.today()
one_week_ago = today - datetime.timedelta(days=7)

In [3]:
# Create connection string
user = "dbadmin"
password = keyring.get_password('docdb_prod_dbadmin', 'dbadmin')
host = "production-documentdb.cluster-cb6kajicuplh.us-east-1.docdb.amazonaws.com"
port = "27017"
options = "tls=true&tlsAllowInvalidCertificates=true&replicaSet=rs0&readPreference=secondaryPreferred&retryWrites=false"
cxn_string_template= "mongodb://{}:{}@{}:{}/?{}"
cxn_string_prod = cxn_string_template.format(user, password, host, port, options)


# Get past week's new products

In [4]:
# Get past week's new products
client = MongoClient(cxn_string_prod)
db = client['product-prod']
results_ = list(db.PdwProductCore.find({'createTimestamp': {'$gte': one_week_ago}}, 
                                       ('productGeneral.cusip', 'productGeneral.isin')))
cusips = [i['productGeneral']['cusip'] for i in results_ if 'cusip' in i['productGeneral']]
isins = [i['productGeneral']['isin'] for i in results_ if 'isin' in i['productGeneral']]

# Get BMO products added within first 5 pages

In [5]:
# Setup for bmo
bmo_act_dict = {}
chrome_path = r"/opt/homebrew/bin/chromedriver"
driver = webdriver.Chrome(chrome_path)
url = 'https://www.bmonotes.com/Type/PPNs#active'
# Get first page
page = driver.get(url)
time.sleep(2)
bmo_act_dict[0] = pd.read_html(driver.page_source)[1]
# Get remaining 4 pages
for num in range(2, 6):
    driver.find_element_by_id("DataTables_Table_1_next").click()
    time.sleep(2)
    bmo_act_dict[num] = pd.read_html(driver.page_source)[1]
driver.close()
# Combine the dataframes
bmo_active_products = pd.concat([bmo_act_dict[k] for k in bmo_act_dict.keys()], ignore_index=True)
# Create pdw cusip for comparison
bmo_active_products['pdwCusip'] = [
    'CA' + i if len(i) == 7 else 'C' + i for i in bmo_active_products['JHN Code / Cusip']
]
# Filter out any products already present in pdw
new_bmo_active_products = bmo_active_products[~bmo_active_products['JHN Code / Cusip'].isin(cusips) | 
                                              bmo_active_products['JHN Code / Cusip'].isin(isins)]

In [6]:
new_bmo_active_products

,Note Name,JHN Code / Cusip,Issue Date,Currency,Term,Maturity Date,Current Bid Price,pdwCusip
0,Bank of Montreal Solactive Canada Bank 40 AR I...,JHN2865,"Aug 8, 2022",CAD,7.00,"Aug 8, 2029",$99.72,CAJHN2865
1,Bank of Montreal Canadian Financials Growth Pr...,JHN2839,"Jul 29, 2022",CAD,5.00,"Jul 29, 2027",$99.76,CAJHN2839
2,Bank of Montreal Canadian Financials Growth Pr...,JHN2840,"Jul 29, 2022",CAD,3.75,"Apr 29, 2026",$99.75,CAJHN2840
3,Bank of Montreal Canadian Financials Growth Pr...,JHN2841,"Jul 29, 2022",CAD,3.00,"Jul 29, 2025",$99.66,CAJHN2841
4,Bank of Montreal Solactive Canada Bank 40 AR I...,JHN2837,"Jul 29, 2022",CAD,7.00,"Jul 30, 2029",$99.24,CAJHN2837
5,Bank of Montreal Solactive Canada Bank 40 AR I...,JHN2838,"Jul 29, 2022",CAD,7.00,"Jul 30, 2029",$99.25,CAJHN2838
6,Bank of Montreal Canadian Equity Principal Pro...,JHN2835,"Jul 28, 2022",CAD,4.75,"Apr 28, 2027",$99.17,CAJHN2835
7,Bank of Montreal Canadian Equity Principal Pro...,JHN2836,"Jul 28, 2022",CAD,4.00,"Jul 28, 2026",$99.32,CAJHN2836
8,Bank of Montreal Canadian Equity AutoCallable ...,JHN2855,"Jul 27, 2022",CAD,7.00,"Jul 27, 2029",$99.93,CAJHN2855
9,Bank of Montreal Solactive Canada Bank 40 AR I...,JHN2852,"Jul 27, 2022",CAD,6.99,"Jul 23, 2029",$100.57,CAJHN2852
